In [1]:
#####Hierarchical Geographical Modeling of User Locationsfrom Social Media Posts#####
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import scipy
from scipy import stats
from scipy import special
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *

#np.random.seed(98537)

In [78]:
####データの生成####
##データの設定
levels = 2
k1 = 5
k2 = np.array([4, 4, 5, 5, 6])
topic = 15
hh = 3000
v = 1000   #語彙数
Lambda = np.random.gamma(10.0, 1/0.25, hh)
pt = np.random.poisson(Lambda, hh); pt[pt < 10] = 10
N = np.sum(pt)
w = np.random.poisson(np.random.gamma(10.0, 1/0.5, N)); w[w < 5] = 5
f = np.sum(w)

In [79]:
##IDとインデックスを作成
#IDを作成
u_id = np.repeat(np.arange(hh), pt)
d_id = np.repeat(np.arange(N), w)

#インデックスを作成
u_list = [i for i in range(hh)]
d_list = [i for i in range(N)]
for i in range(hh):
    u_list[i] = np.array(np.where(u_id==i)[0], dtype="int")
for i in range(N):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")

In [105]:
##ノード分布を生成
#グローバルノード分布を生成
G1 = np.random.dirichlet(np.repeat(3.0, k1), 1).reshape(-1)
G2 = [np.random.dirichlet(np.repeat(2.5, k2[j]), 1).reshape(-1)  for j in range(k1)]

#ローカルノード分布を生成
alpha1 = 2.0
theta1 = np.random.dirichlet(alpha1*G1, hh)
theta2 = [np.random.dirichlet(alpha1*G2[j], hh) for j in range(k1)]

##モデルパラメータを生成
#トピック分布を生成
alpha2 = np.repeat(0.1, topic)
gamma = [np.random.dirichlet(alpha2, k2[j]) for j in range(k1)]

#location分布を生成
longitude = np.array((0, 10)); latitude = np.array((0, 10))
mu = [j for j in range(k1)]
Cov = [j for j in range(k1)]
for i in range(k1):
    mu0 = np.zeros((k2[i], 2))
    Cov0 = np.zeros((2, 2, k2[i]))
    for j in range(k2[i]):
        mu0[j, ] = np.append(np.random.uniform(longitude[0], longitude[1], 1), np.random.uniform(latitude[0], latitude[1], 1)) 
        cov_temp = np.diag(np.random.uniform(0.025, 0.5, 2))
        value = np.random.uniform(-0.7, 0.7, 1) * np.prod(np.sqrt(np.diag(cov_temp)))
        cov_temp[0, 1] = value; cov_temp[1, 0] = value
        Cov0[:, :, j] = cov_temp
    mu[i] = mu0
    Cov[i] = Cov0
    
#単語分布を生成



[array([[5.86080894e-23, 2.53490713e-02, 2.73037747e-02, 1.04753701e-02,
         1.53314220e-05, 1.34928665e-09, 4.36788036e-04, 1.22401535e-06,
         8.42754155e-01, 3.56902258e-12, 7.02938877e-02, 2.07466639e-15,
         8.00092293e-14, 2.33703720e-02, 2.42432159e-08],
        [3.06569450e-02, 2.45990934e-04, 3.18617911e-05, 6.69602838e-02,
         8.25980411e-08, 6.70890330e-03, 5.41110127e-04, 1.01897229e-01,
         2.84240840e-03, 2.34031292e-07, 7.71583944e-02, 2.86597522e-14,
         1.93062297e-02, 6.93650321e-01, 5.53534814e-09],
        [2.42925833e-01, 1.49145677e-02, 9.44650750e-08, 2.03763840e-01,
         5.35766223e-18, 4.22489130e-05, 1.51513127e-06, 3.49755123e-03,
         5.52166456e-11, 3.84258304e-04, 4.92167778e-01, 2.60950899e-02,
         1.57629281e-02, 4.31244569e-04, 1.30503739e-05],
        [1.26933577e-01, 2.67028115e-10, 7.59427021e-08, 7.50225300e-02,
         1.16743087e-09, 1.79741607e-02, 1.92450205e-04, 4.04517967e-10,
         6.56430000e-05

In [101]:
Cov[0][:, :, 2]

array([[ 0.14430379, -0.05743551],
       [-0.05743551,  0.13773579]])

In [77]:
theta1[:, 1]

array([0.08430941, 0.5333448 , 0.11757715, ..., 0.01293244, 0.37239515,
       0.15145536])